In [ ]:
import os
import cv2
import shutil
import numpy as np
from skimage.restoration import denoise_tv_chambolle

# Gaussian Blur function
def gaussian_blur(image, kernel_size=(5, 5), sigmaX=0):
    return cv2.GaussianBlur(image, kernel_size, sigmaX)

# Bilateral Filter function
def bilateral_filter(image, d=9, sigma_color=75, sigma_space=75):
    return cv2.bilateralFilter(image, d, sigma_color, sigma_space)

# Median Filter function
def median_filter(image, kernel_size=3):
    return cv2.medianBlur(image, kernel_size)

# Anisotropic Diffusion function
def anisotropic_diffusion(image, n_iter=50, delta_t=0.1, kappa=100):
    smoothed_image = denoise_tv_chambolle(image, n_iter=n_iter, step=delta_t, eps=kappa)
    return np.clip(smoothed_image, 0, 255).astype(np.uint8)

# Non-Local Means Filter function
def non_local_means(image, h=10, h_for_color=10, template_window_size=5, search_window_size=21):
    return cv2.fastNlMeansDenoisingColored(image, None, h, h_for_color, template_window_size, search_window_size)

# Gradient-based method (Gradient Inverse Weighted Interpolation) function
def gradient_inverse_weighted_interpolation(image, alpha=0.2, threshold=30):
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)

    smoothed_image = image.copy()
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            if grad_magnitude[i, j] > threshold:
                continue
            weight = 1.0 - grad_magnitude[i, j] / threshold
            new_value = weight * image[i - 1, j] + weight * image[i + 1, j] + weight * image[i, j - 1] + weight * image[i, j + 1]
            smoothed_image[i, j] = new_value

    smoothed_image = (smoothed_image * alpha + image * (1 - alpha)).astype(np.uint8)
    return smoothed_image

# Function to apply an edge smoothing technique and save the result to a folder
def apply_edge_smoothing(original_folder, output_folder, smoothing_function, **params):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(original_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(original_folder, filename)
            original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Apply the edge smoothing technique
            smoothed_image = smoothing_function(original_image, **params)

            # Save the smoothed image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, smoothed_image)

# Define the paths for the original segmented images folder and output folders for each technique
original_images_folder = "path/to/original_images_folder"
output_gaussian_folder = "path/to/output/gaussian"
output_bilateral_folder = "path/to/output/bilateral"
output_median_folder = "path/to/output/median"
output_anisotropic_folder = "path/to/output/anisotropic"
output_non_local_means_folder = "path/to/output/non_local_means"
output_gradient_based_folder = "path/to/output/gradient_based"

# Apply different edge smoothing techniques to the original images
apply_edge_smoothing(original_images_folder, output_gaussian_folder, gaussian_blur, kernel_size=(5, 5), sigmaX=0)
apply_edge_smoothing(original_images_folder, output_bilateral_folder, bilateral_filter, d=9, sigma_color=75, sigma_space=75)
apply_edge_smoothing(original_images_folder, output_median_folder, median_filter, kernel_size=3)
apply_edge_smoothing(original_images_folder, output_anisotropic_folder, anisotropic_diffusion, n_iter=50, delta_t=0.1, kappa=100)
apply_edge_smoothing(original_images_folder, output_non_local_means_folder, non_local_means, h=10, h_for_color=10, template_window_size=5, search_window_size=21)
apply_edge_smoothing(original_images_folder, output_gradient_based_folder, gradient_inverse_weighted_interpolation, alpha=0.2, threshold=30)
